# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5, kgrid=(1, 1, 1))

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.124688e+02     1.560195e+00
 * time: 0.12476396560668945
     1     1.097085e+01     9.104558e-01
 * time: 1.4601819515228271
     2    -1.145221e+01     1.004775e+00
 * time: 1.528130054473877
     3    -3.395551e+01     8.396454e-01
 * time: 1.618600845336914
     4    -4.736370e+01     6.728611e-01
 * time: 1.7180979251861572
     5    -5.697525e+01     2.446134e-01
 * time: 1.793949842453003
     6    -5.980586e+01     1.958807e-01
 * time: 1.8516249656677246
     7    -6.092854e+01     6.577497e-02
 * time: 1.9300549030303955
     8    -6.140944e+01     4.435834e-02
 * time: 1.990488052368164
     9    -6.169865e+01     3.471013e-02
 * time: 2.0513789653778076
    10    -6.186340e+01     3.303808e-02
 * time: 2.1116600036621094
    11    -6.201330e+01     2.482539e-02
 * time: 2.1721270084381104
    12    -6.210004e+01     2.079766e-02
 * time: 2.241644859313965
    13    -6.214071e+01     1.363802e-02
 * time: 2.30229783058166

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671074
    AtomicLocal         -18.8557695
    AtomicNonlocal      14.8522652
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485380 
    Xc                  -19.3336823

    total               -62.261666460151
